In [ ]:
! pip install pytelegrambotapi requests

In [ ]:
import telebot

In [ ]:
import requests

In [ ]:
import logging

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

In [ ]:
API_TOKEN = '7119724103:AAHyuxSL7yKDIu_mmxTfm596OzTZV2Z2yDo'
bot = telebot.TeleBot(API_TOKEN)

In [ ]:
WEATHER_API_KEY = 'afac80a4a23d148c59bd20c85fe3f18f'

In [ ]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Olá! Eu sou o WeatherBot. Você pode me pedir informações sobre o clima.\n"
                          "Use /help para ver os comandos disponíveis.")

@bot.message_handler(commands=['help'])
def send_help(message):
    bot.reply_to(message, "/start - Iniciar interação com o bot\n"
                          "/weather [cidade] - Obter informações sobre o clima da cidade\n"
                          "Basta digitar qualquer outra coisa para iniciar uma nova conversa.")

@bot.message_handler(commands=['weather'])
def get_weather(message):
    cidade = ' '.join(message.text.split()[1:])
    if cidade:
        try:
            weather_info = fetch_weather(cidade)
            if weather_info:
                bot.reply_to(message, weather_info)
            else:
                bot.reply_to(message, 'Não consegui obter informações sobre o clima dessa cidade.')
        except Exception as e:
            logging.error(f'Erro ao obter informações do clima: {e}')
            bot.reply_to(message, f'Ocorreu um erro: {e}')
    else:
        bot.reply_to(message, 'Por favor, forneça o nome da cidade após o comando /weather.')

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    bot.reply_to(message, 'Use /weather [cidade] para obter informações sobre o clima.')

def fetch_weather(city):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': WEATHER_API_KEY,
        'units': 'metric',
        'lang': 'pt'
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        city_name = data['name']
        country = data['sys']['country']
        temp = data['main']['temp']
        description = data['weather'][0]['description']
        humidity = data['main']['humidity']
        wind_speed = data['wind']['speed']

        weather_info = (f"Clima em {city_name}, {country}:\n"
                        f"Temperatura: {temp}°C\n"
                        f"Descrição: {description}\n"
                        f"Umidade: {humidity}%\n"
                        f"Velocidade do Vento: {wind_speed} m/s")
        return weather_info
    else:
        logging.error(f'Erro na chamada da API: {response.status_code}')
        return None

In [ ]:
bot.polling()